## **Estructura de sub-etapas del procesamiento del habla:**

---

```
├── Input Layer
│   ├── Audio Capture
│   └── ASR (Speech-to-Text)
├── NLP Layer
│   ├── Text Preprocessing
│   ├── Intent Detection
│   ├── Entity Recognition
│   └── Sentiment Analysis
├── Dialogue Management
│   ├── State Management
│   ├── Response Generation
│   ├── Response Selection
│   └── Context Handling
├── Output Layer
│   ├── TTS (Text-to-Speech)
│   └── Audio Playback

```


### **Estructura operativa para cada sub-etapa:**

**Selección de técnicas de modelado:** En este caso se utilizara [] para la etapa de reconocimiento de voz, [] para el procesamiento y analisis de texto y [] para la conversion de texto a voz.

**Generación de un diseño de comprobación:** Para elegir el modelo correcto, este devera ser el que pondere mas alto en un promedio de las diferentes metricas de evaluacion del modelo.

**Generación de los modelos:** Se definiran y configuraran los parametros del modelo para pasar a la etapa de ejecucion y descripcion.

**Evaluación del modelo:** Se evaluaran los diferentes modelos de manera individual en busqueda de optimizar sus parametros y escoger la combinacion mas optima entre modelos / parametros individuales.

In [ ]:
# Execute if use colab and you need import files
from google.colab import files

In [ ]:
# Execute if use colab and you need import files
drive.mount('/content/drive')

In [ ]:
# Execute if use colab
%%bash
apt-get update
apt-get install python3
apt-get install -venv
python3 -m venv proc_habla
source proc_habla/bin/activate

In [1]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

## **1. Entrada de Usuario (Reconocimiento del Habla)**

In [47]:
!pip3 install SpeechRecognition
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub

In [80]:
import speech_recognition as sr
import numpy as np
import wave
import pandas as pd
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment
import subprocess

In [81]:
try:
    subprocess.run(["sudo", "apt-get", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en sistemas basados en Debian.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)
try:
    subprocess.run(["brew", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en macOS utilizando Homebrew.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


FFmpeg instalado exitosamente en sistemas basados en Debian.


Running `brew update --auto-update`...
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:86ff822590529e8e9093cdc1702a1d3321678c85347d30f82db4f993db8f9eb1
######################################################################## 100.0%#############                                                     30.3%######################                                        47.6%
==> Pouring portable-ruby-3.3.1.arm64_big_sur.bottle.tar.gz


==> Downloading https://ghcr.io/v2/homebrew/core/ffmpeg/manifests/7.0.1
==> Fetching dependencies for ffmpeg: brotli, giflib, highway, imath, jpeg-turbo, libpng, xz, lz4, zstd, libtiff, little-cms2, openexr, webp, jpeg-xl, libvmaf, aom, aribb24, dav1d, freetype, fontconfig, frei0r, ca-certificates, gmp, libunistring, gettext, libidn2, libtasn1, nettle, p11-kit, openssl@3, libevent, libnghttp2, unbound, gnutls, mpdecimal, readline, sqlite, python@3.12, glib, xorgproto, libxau, libxdmcp, libxcb, libx11, libxext, libxrender, lzo, pixman, cairo, graphite2, icu4c, harfbuzz, lame, fribidi, libunibreak, libass, libbluray, cjson, libmicrohttpd, mbedtls, librist, libsoxr, libssh, libvidstab, libogg, libvorbis, libvpx, opencore-amr, openjpeg, opus, rav1e, libsamplerate, flac, mpg123, libsndfile, rubberband, sdl2, snappy, speex, srt, svt-av1, leptonica, libb2, libarchive, pango, tesseract, theora, x264, x265, xvid, libsodium, zeromq and zimg
==> Downloading https://ghcr.io/v2/homebrew/core/brotli

## **Módulo de Captura de Audio:**

*Bugs*

- Recibo algun id de consulta, cliente? (lo necesito para gestionar la consulta de manera eficiente)

In [71]:
SAMPLE_RATE = 44100  # Tasa de muestreo
DURATION = 10  # Duración de la grabación en segundos
AUDIO_FILES_PATH = "../datos/brutos/audios_proc_habla/"
FILENAME = "output.wav"  # Nombre del archivo de salida

In [73]:
ruta_audio_entrada = AUDIO_FILES_PATH + FILENAME
ruta_audio_entrada_convertido = AUDIO_FILES_PATH + 'waw_conv_' + FILENAME

In [63]:
columns_consultas_df = ['id_consulta', 'id_cliente', 'formato_consulta', 'transcripción_audio', 'entrada_texto', 'productos_detectados']

In [64]:
consultas_df = pd.DataFrame(columns=columns_consultas_df)

In [75]:
def record_audio(audio_file_path = AUDIO_FILES_PATH, filename = FILENAME, duration = DURATION, sample_rate = SAMPLE_RATE):
    print("Grabando...")
    # Grabar audio con 1 canal (mono)
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Esperar a que termine la grabación
    print("Grabación finalizada")
    write(ruta_audio_entrada, sample_rate, recording)
    return filename

In [76]:
def convert_to_wav(input_file = ruta_audio_entrada, output_file = ruta_audio_entrada_convertido):
    sound = AudioSegment.from_file(input_file)
    sound.export(output_file, format="wav")

In [77]:
def chatbot():
    record_audio()

In [42]:
chatbot()

Grabando...
Grabación finalizada
../datos/brutos/audios_proc_habla/output.wav


In [78]:
# Uso de la función convert_to_wav
convert_to_wav(ruta_audio_entrada, ruta_audio_entrada_convertido)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

## **Reconocimiento del Habla (ASR - Automatic Speech Recognition):**

In [43]:
# Crear un objeto Recognizer
recognizer = sr.Recognizer()

In [44]:
# Abrir el archivo de audio
def transcribe_audio(audio_lang = 'es-ES'):
    ruta_audio = AUDIO_FILES_PATH + FILENAME
    with sr.AudioFile(ruta_audio) as source:
        # Escuchar el audio (en inglés)
        audio = recognizer.record(source)

        # Utilizar Google Speech Recognition para transcribir el audio
        try:
            text = recognizer.recognize_google(audio, audio_lang)
            print("Transcripción: ", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print("Error al solicitar resultados del servicio Google Speech Recognition; {0}".format(e))

In [45]:
transcribe_audio()

ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [ ]:
# Transcribir y mostrar el resultado
text = transcribe_audio(audio_file_path, audio_lang)

## **2. Procesamiento del Lenguaje Natural (NLP)**

### **Preprocesamiento de Texto:**



**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

**Tokenización**

**Eliminación de stop words.**


**Lematización y stemming**

### **Análisis de Texto:**


Análisis de Sentimiento: Determinar la emoción o tono del texto.


Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


Extracción de Entidades (Entity Recognition): Extraer información relevante del texto, como nombres, fechas, ubicaciones, etc.

## **3. Gestión del Diálogo**

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

## 4. Salida de Usuario (Text-to-Speech)


### Conversión de Texto a Voz (TTS): Utilizar servicios como Google Text-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


### Reproducción de Audio: Entregar la respuesta de voz al usuario.